# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Files to read
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Merge two data sets
df_school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
df_school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#create array of unique school names
unique_school_names = df_school_data_complete['school_name'].unique()

#gives the length of unique school names to give us how many schools
school_count = len(unique_school_names)

#district student count
dist_student_count = df_school_data_complete['size'].sum()

#student count from student file (to verify with district student count)
total_student_rec = df_school_data_complete['student_name'].count()

#total budget
total_budget = df_school_data_complete['budget'].sum()

#calculations for number and % passing reading
num_passing_reading = df_school_data_complete.loc[df_school_data_complete['reading_score'] >= 70]['reading_score'].count()
perc_pass_reading = num_passing_reading/total_student_rec

#calculations for number and % passing math
num_passing_math = df_school_data_complete.loc[df_school_data_complete['math_score'] >= 70]['math_score'].count()
perc_pass_math = num_passing_math/total_student_rec

#average math score calculation
avg_math_score = df_school_data_complete['math_score'].mean()

#average reading score calculation
avg_reading_score = df_school_data_complete['reading_score'].mean()

#Overall Passing Rate Calculations
overall_pass = df_school_data_complete.loc[(df_school_data_complete['math_score'] >= 70) & (df_school_data_complete['reading_score'] >= 70)]['student_name'].count()/total_student_rec

# district dataframe from dictionary

district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [dist_student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading":[perc_pass_reading],
    "% Passing Math": [perc_pass_math],
    "Overall Passing Rate": [overall_pass]

})

#store as different df to change order
dist_sum = district_summary[["Total Schools", 
                             "Total Students", 
                             "Total Budget", 
                             "Average Reading Score", 
                             "Average Math Score", 
                             '% Passing Reading', 
                             '% Passing Math', 
                             'Overall Passing Rate']]

#format cells
dist_sum.style.format({"Total Budget": "${:,.2f}", 
                       "Average Reading Score": "{:.1f}", 
                       "Average Math Score": "{:.1f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"})


,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,130551930,"$82,932,329,558.00",81.9,79.0,85.8%,75.0%,65.2%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:
grouped_schools = df_school_data_complete.groupby("school_name")

# add data that's easier to get from school_data
school_summary = pd.DataFrame({"School Name":school_data["school_name"], "School Type":school_data["type"],
                             "Total Budget":school_data["budget"]})
school_summary = school_summary.set_index("School Name")
school_summary = school_summary.sort_values("School Name")

# do calculations on data from groupby object
school_summary["Total Students"] = grouped_schools["Student ID"].count()
school_summary["Budget Per Student"] = school_summary["Total Budget"] / school_summary["Total Students"]
school_summary["Average Math Score"] = round(grouped_schools["math_score"].mean(), 2)
school_summary["Average Reading Score"] = round(grouped_schools["reading_score"].mean(), 2)

# calculate percentages of students passing using apply
school_summary["Percent Passing Math"] = round((grouped_schools.apply(lambda x: (x["math_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]) * 100, 2)
school_summary["Percent Passing Reading"] = round((grouped_schools.apply(lambda x: (x["reading_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]) * 100, 2)
school_summary["Overall Passing Rate"] = round((school_summary["Percent Passing Math"] + 
                                               school_summary["Percent Passing Reading"]) / 2, 2)

school_summary = school_summary.sort_values("Overall Passing Rate", ascending=False)

# add formatting
school_summary["Total Budget"] = school_summary["Total Budget"].map('${:,}'.format)

# could also format the following columns, but we need the values for analyses later

# school_summary["Percent Passing Math"] = school_summary["Percent Passing Math"].map('{}%'.format)
# school_summary["Percent Passing Reading"] = school_summary["Percent Passing Reading"].map('{}%'.format)
# school_summary["Overall Passing Rate"] = school_summary["Overall Passing Rate"].map('{}%'.format)
# school_summary["Budget Per Student"] = school_summary["Budget Per Student"].map('${:}'.format)
# school_summary["Total Students"] = school_summary["Total Students"].map('{:,}'.format)

school_summary.head()

,School Type,Total Budget,Total Students,Budget Per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356",1858,582.0,83.06,83.98,94.13,97.04,95.58
Thomas High School,Charter,"$1,043,130",1635,638.0,83.42,83.85,93.27,97.31,95.29
Pena High School,Charter,"$585,858",962,609.0,83.84,84.04,94.59,95.95,95.27
Griffin High School,Charter,"$917,500",1468,625.0,83.35,83.82,93.39,97.14,95.26
Wilson High School,Charter,"$1,319,574",2283,578.0,83.27,83.99,93.87,96.54,95.21


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [4]:
school_summary = school_summary.sort_values("Overall Passing Rate", ascending=True)

school_summary.head()

,School Type,Total Budget,Total Students,Budget Per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"$2,547,363",3999,637.0,76.84,80.74,66.37,80.22,73.30
Figueroa High School,District,"$1,884,411",2949,639.0,76.71,81.16,65.99,80.74,73.36
Huang High School,District,"$1,910,635",2917,655.0,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,"$3,094,650",4761,650.0,77.07,80.97,66.06,81.22,73.64
Ford High School,District,"$1,763,916",2739,644.0,77.10,80.75,68.31,79.30,73.81


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
ninth = df_school_data_complete.loc[df_school_data_complete["grade"] == "9th"]
tenth = df_school_data_complete.loc[df_school_data_complete["grade"] == "10th"]
eleventh = df_school_data_complete.loc[df_school_data_complete["grade"] == "11th"]
twelfth = df_school_data_complete.loc[df_school_data_complete["grade"] == "12th"]

math_ninth = ninth.groupby("school_name")["math_score"].mean()
math_tenth = tenth.groupby("school_name")["math_score"].mean()
math_eleventh = eleventh.groupby("school_name")["math_score"].mean()
math_twelfth = twelfth.groupby("school_name")["math_score"].mean()

math_grades = pd.DataFrame({"9th":math_ninth, "10th":math_tenth, 
                             "11th":math_eleventh, "12th":math_twelfth})

math_grades = round(math_grades, 4)

math_grades

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.0837,76.9968,77.5156,76.4922
Cabrera High School,83.0947,83.1545,82.7656,83.2775
Figueroa High School,76.4030,76.5400,76.8843,77.1514
Ford High School,77.3613,77.6723,76.9181,76.1800
Griffin High School,82.0440,84.2291,83.8421,83.3562
Hernandez High School,77.4385,77.3374,77.1360,77.1866
Holden High School,83.7874,83.4298,85.0000,82.8554
Huang High School,77.0273,75.9087,76.4466,77.2256
Johnson High School,77.1879,76.6911,77.4917,76.8632


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
reading_ninth = ninth.groupby("school_name")["reading_score"].mean()
reading_tenth = tenth.groupby("school_name")["reading_score"].mean()
reading_eleventh = eleventh.groupby("school_name")["reading_score"].mean()
reading_twelfth = twelfth.groupby("school_name")["reading_score"].mean()

reading_grades = pd.DataFrame({"9th":reading_ninth, "10th":reading_tenth, 
                             "11th":reading_eleventh, "12th":reading_twelfth})

reading_grades = round(reading_grades, 4)

reading_grades

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3032,80.9072,80.9456,80.9125
Cabrera High School,83.6761,84.2532,83.7884,84.2880
Figueroa High School,81.1986,81.4089,80.6403,81.3849
Ford High School,80.6327,81.2627,80.4036,80.6623
Griffin High School,83.3692,83.7069,84.2881,84.0137
Hernandez High School,80.8669,80.6601,81.3961,80.8571
Holden High School,83.6772,83.3246,83.8155,84.6988
Huang High School,81.2903,81.5124,81.4175,80.3060
Johnson High School,81.2607,80.7734,80.6160,81.2276


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [8]:
school_summary["Spending Ranges"] = pd.cut(school_summary["Budget Per Student"], spending_bins, labels=group_names)

spending_groups = school_summary.loc[:, ["Spending Ranges","Average Math Score", "Average Reading Score",
                                     "Percent Passing Math", "Percent Passing Reading",
                                     "Overall Passing Rate"]].groupby("Spending Ranges")
spending_groups.mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Spending Ranges,,,,,
<$585,83.452500,83.935000,93.460000,96.610000,95.035000
$585-615,83.600000,83.885000,94.230000,95.900000,95.065000
$615-645,79.078333,81.891667,75.668333,86.106667,80.888333
$645-675,76.996667,81.026667,66.163333,81.133333,73.650000


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [10]:
school_summary["Size Ranges"] = pd.cut(school_summary["Total Students"], size_bins, labels=group_names)

size_groups = school_summary.loc[:, ["Size Ranges","Average Math Score", "Average Reading Score",
                                     "Percent Passing Math", "Percent Passing Reading",
                                     "Overall Passing Rate"]].groupby("Size Ranges")

size_groups.mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Size Ranges,,,,,
Small (<1000),83.820,83.92500,93.55000,96.10000,94.8250
Medium (1000-2000),83.374,83.86800,93.59800,96.79000,95.1920
Large (2000-5000),77.745,81.34375,69.96375,82.76625,76.3675


## Scores by School Type

* Perform the same operations as above, based on school type.

In [11]:
district_groups = school_summary.loc[:, ["School Type", "Average Math Score", "Average Reading Score",
                                     "Percent Passing Math", "Percent Passing Reading",
                                     "Overall Passing Rate"]].groupby("School Type")
district_groups.mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.472500,83.897500,93.620000,96.586250,95.102500
District,76.955714,80.965714,66.548571,80.798571,73.675714
